# Bar
Polarsも棒グラフに対応している.

````
```Note
MultiindexはPandasにのみ存在する機能のため,層化棒グラフの描画はPandasでのみ可能です.
```
````

In [14]:
import hvplot.pandas  # noqa

import hvplot.polars  # noqa
import polars as pl

## Introduction
``bar`` プロットは、**カテゴリデータ**を、それが表す**数値**に比例した高さを持つ長方形の棒で表します。
X 軸はカテゴリを表し、Y 軸は数値スケールを表します。 バーは同じ幅なので、データを瞬時に比較できます。

In [15]:
pl.DataFrame({
    "framework": ["hvPlot", "HoloViews", "Panel"], 
    "stars": [700, 2400, 2600]
}).hvplot.bar(x = "framework", y = "stars", color = "gold", title = "Bar Plot of Github Stars", ylabel = "⭐")

:Bars   [framework]   (stars)

## Data
データをインポートしてみましょう。

````
```Note
Multiindexの例と矛盾しないように,グラフを描画する時にPandasデータフレームをPolarsデータフレームに変換します.
```
````

In [16]:
from bokeh.sampledata.autompg import autompg_clean as autompg

autompg.head()

,mpg,cyl,displ,hp,weight,accel,yr,origin,name,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,North America,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,North America,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,North America,ford torino,ford


``yr`` カテゴリ値ごとに 1 行を持つ長い形式のデータを定義します。

In [17]:
autompg_long_form = autompg.groupby("yr").mean(numeric_only = True).reset_index()
autompg_long_form.head()

,yr,mpg,cyl,displ,hp,weight,accel
0,70,17.689655,6.758621,281.413793,147.827586,3372.793103,12.948276
1,71,21.111111,5.629630,213.888889,107.037037,3030.592593,15.000000
2,72,18.714286,5.821429,218.375000,120.178571,3237.714286,15.125000
3,73,17.100000,6.375000,256.875000,130.475000,3419.025000,14.312500
4,74,22.769231,5.230769,170.653846,94.230769,2878.038462,16.173077


複数のカテゴリを表すマルチインデックスを持つデータセットを定義します

In [18]:
# 1980年までの地域別の車の量的変数の平均値を集計する
autompg_multi_index = autompg.query("yr<=80").groupby(['yr', 'origin']).mean(numeric_only = True)
autompg_multi_index.head()

mpg       cyl       displ          hp  weight  \
yr origin                                                               
70 Asia           25.500000  4.000000  105.000000   91.500000  2251.0   
   Europe         25.200000  4.000000  107.800000   86.200000  2309.2   
   North America  15.272727  7.636364  336.909091  166.954545  3716.5   
71 Asia           29.500000  4.000000   88.250000   79.250000  1936.0   
   Europe         28.750000  4.000000   95.000000   74.000000  2024.0   

                      accel  
yr origin                    
70 Asia           14.750000  
   Europe         16.500000  
   North America  11.977273  
71 Asia           16.375000  
   Europe         16.750000

広い形式のデータ、つまり ```origin``` のようなカテゴリを表す複数の列を定義します。

In [19]:
autompg_wide = autompg_multi_index.reset_index().pivot(index='yr', columns='origin', values='mpg')
autompg_wide.head()

origin,Asia,Europe,North America
yr,,,
70,25.500000,25.20,15.272727
71,29.500000,28.75,17.736842
72,24.200000,22.00,16.277778
73,20.000000,24.00,15.034483
74,29.333333,27.00,18.142857


## Basic Bar Plots
```x``` 引数を使用してカテゴリカルな x 値を指定し、```y``` 引数を使用して数値の y 値を指定すると、長い形式のデータをプロットできます。

In [20]:
pl.from_pandas(autompg_long_form).hvplot.bar(x="yr", y="mpg", width=1000)

:Bars   [yr]   (mpg)

```x``` 引数を指定しない場合は、インデックスが使用されます。

In [21]:
pl.from_pandas(autompg_long_form).hvplot.bar(y = "mpg", width = 1000)

:Bars   [index]   (mpg)

インデックスが ```MultiIndex``` の場合、X 軸はインデックスに含まれる複数のカテゴリを表し、外側のインデックス レベルが外側のカテゴリとして表示されます。

In [23]:
# マルチインデックスはPandasの機能
autompg_multi_index.hvplot.bar(width = 1000, rot = 90)

:Bars   [yr,origin]   (mpg)

代わりに、```stacked``` を ```True``` に設定することで、ネストされたインデックス/カテゴリの値 (この例では原点) を y 軸に積み重ねることができます。

In [31]:
autompg_multi_index.hvplot.bar(stacked = True, width = 1000, legend = "top_left", height = 500)

:Bars   [yr,origin]   (mpg)

データが **ワイド形式**の場合に複数のカテゴリを X 軸にプロットするには、列のリストを ```y``` に提供する必要があります。

また、ワイドフォームデータの値をスタックすることもできます。

In [32]:
pl.from_pandas(autompg_wide).hvplot.bar(y = ['Asia', 'Europe', 'North America'], ylabel = "mpg", stacked = True, width = 1000, legend = "top_left", height = 500)

:Bars   [index,Variable]   (value)

## Colorful Bar Plots
```color``` 引数を使用して ```bar``` の色を制御できます。 列の名前、色の名前、または色のリストを受け入れます。

以下は、単一の名前付き色を使用した例です。


In [33]:
pl.from_pandas(autompg_long_form).hvplot.bar(x = "yr", y = "mpg", color = "teal", width = 1000)

:Bars   [yr]   (mpg)

以下は色のリストを使用した例です。

In [27]:
autompg_wide.hvplot.bar(y = ['Asia', 'Europe', 'North America'], width = 1000, ylabel = "mpg", color = ["#ba2649", "#ffa7ca", "#1a6b54"], rot = 90)

:Bars   [yr,origin]   (value)

以下は列の名前を使用した例です。

In [34]:
pl.from_pandas(autompg_long_form).hvplot.bar(y = 'mpg', color = "weight", colorbar = True, clabel = "Weight", cmap = "bmy", width = 1000)

:Bars   [index]   (mpg,weight)